In [1]:
import pandas as pd
import re

In [2]:
arlette = pd.read_csv("../datos_extraidos/casas_arlette.csv")
arlette = arlette.drop_duplicates()

# Elijo unicamente los inmuebles que son casas
arlette = arlette.query("property_type=='Casa'").reset_index(drop="first")

# Elimino los acentos
arlette["address"] = arlette.address.apply(lambda x: x.lower()\
                                           .replace("á", "a")\
                                            .replace("é", "e")\
                                            .replace("í", "i")\
                                            .replace("ó", "o")\
                                            .replace("ú", "u"))

In [3]:
arlette.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          20 non-null     object 
 1   seller         20 non-null     object 
 2   property_type  20 non-null     object 
 3   address        20 non-null     object 
 4   price          19 non-null     float64
 5   bedrooms       20 non-null     float64
 6   bathrooms      19 non-null     float64
 7   built_area     19 non-null     float64
 8   land_area      19 non-null     float64
 9   description    20 non-null     object 
 10  parking        20 non-null     float64
dtypes: float64(6), object(5)
memory usage: 1.8+ KB


In [4]:
arlette = arlette.dropna()
arlette = arlette.reset_index(drop="first")

In [5]:
# Un valor de 8 para parking es atípico
arlette.parking.unique()

array([2., 3., 8., 4., 5.])

In [6]:
arlette.query("parking>5")

,title,seller,property_type,address,price,bedrooms,bathrooms,built_area,land_area,description,parking
10,Hermosa propiedad en renta en San Felipe del A...,arlette,Casa,san felipe del agua,50000.0,3.0,2.0,350.0,800.0,En renta! Hermosa propiedad en San Felipe del ...,8.0


In [7]:
# Aunque el dato extraído marca 8, la descripción menciona 4 lugares de parking
arlette.description[9]

'En venta, hermosa residencia en Santa Lucia del Camino, a solo unas cuadras del Palacio Municipal, con fácil acceso desde Avenida Ferrocarril y Camino Nacional.,Superficie de terreno de 475 metros cuadrados, y 520 metros cuadrados de construcción en dos plantas.,Entrando a la casa, un hermoso jardín nos recibe y da acceso a la cochera techada para dos autos.,Un hermoso corredor rodea la sala- con precioso bar-, el comedor, el estudio, y dos amplísimas recámaras que cuentan con baño propio, todo con vista a una gran piscina ovalada, totalmente equipada, rodeada por preciosas jardineras, zona de asador, y baños que dan servicio al area de la piscina.,La cocina integral es totalmente equipada, y da acceso a el area de servicio que cuenta con lavandería y cuarto de servicio con baño completo.,En la planta alta, encontramos la recámara principal, que cuenta con precioso vestidor, y baño completo con sauna. También en esta planta encontramos otra amplísima recámara- también con baño propio,

In [8]:
# Se corrige el dato
arlette.loc[arlette.query("parking>5").index[0], "parking"] = 4

In [9]:
# Los valores ahora son correctos
arlette.parking.unique()

array([2., 3., 4., 5.])

In [10]:
arlette.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          16 non-null     object 
 1   seller         16 non-null     object 
 2   property_type  16 non-null     object 
 3   address        16 non-null     object 
 4   price          16 non-null     float64
 5   bedrooms       16 non-null     float64
 6   bathrooms      16 non-null     float64
 7   built_area     16 non-null     float64
 8   land_area      16 non-null     float64
 9   description    16 non-null     object 
 10  parking        16 non-null     float64
dtypes: float64(6), object(5)
memory usage: 1.5+ KB


### Creando la columna zip_code

In [11]:
# Leo la tabla de códigos postales
tabla_CP = pd.read_html("https://codigo-postal.co/mexico/oaxaca/oaxaca-de-juarez/")
tabla_CP = tabla_CP[0]

In [12]:
# Seleccionar las columas que me sirven 
tabla_CP = tabla_CP.iloc[:, :2]
tabla_CP = tabla_CP.drop_duplicates()

In [13]:
# Guardar en listas las colonias y sus C.P.
cp = list(tabla_CP.iloc[:, 0])
direccion = list(tabla_CP.iloc[:, 1])

In [14]:
# Crear un diccionario y Eliminar acentos de los nombres de las direcciones
cp_dict = {}
for i, j in zip(direccion, cp):
    cp_dict[i.lower()\
        .replace("á", "a")\
        .replace("é", "e")\
        .replace("í", "i")\
        .replace("ó", "o")\
        .replace("ú", "u")] = j

In [15]:
# Eliminar palabras que entorpecen la búsqueda del C.P.
arlette["address"] = arlette.address.apply(
    lambda x: x.replace("agencia", "").replace("municipal", "").strip())

In [16]:
# Creo una lista con los datos de zip code para cada propiedad
cp_column = []
for i, d in zip(arlette.index, arlette.address):
    regex = re.compile(fr"(.{{0,20}}{d}.{{0,20}})", re.IGNORECASE)
    location_name = list(filter(lambda x: regex.findall(x), cp_dict.keys()))

    try:
        cp_column.append(cp_dict[location_name[0]])
    except:
        cp_column.append(None)

In [17]:
# Creo la columna zip_code con los valores obtenidos de la iteración
arlette["zip_code"] = cp_column

In [18]:
arlette.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          16 non-null     object 
 1   seller         16 non-null     object 
 2   property_type  16 non-null     object 
 3   address        16 non-null     object 
 4   price          16 non-null     float64
 5   bedrooms       16 non-null     float64
 6   bathrooms      16 non-null     float64
 7   built_area     16 non-null     float64
 8   land_area      16 non-null     float64
 9   description    16 non-null     object 
 10  parking        16 non-null     float64
 11  zip_code       15 non-null     float64
dtypes: float64(7), object(5)
memory usage: 1.6+ KB


In [19]:
arlette = arlette.drop_duplicates()
arlette = arlette.dropna(ignore_index=True)

In [20]:
arlette.to_csv("datos_arlette.csv", index=False)